In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [3]:
from pyspark.sql import types
from pyspark.sql import functions as F

In [4]:
import os
import pandas as pd
import pendulum as pdl

# inputs
- city

In [5]:
city = 'chicago'

In [6]:
gcs_bkt = os.getenv('GCP_GCS_BUCKET')

In [7]:
jar_path = os.getenv('JAR_FILE_LOC')
creds_path = '/.google/credentials/' + os.getenv('GOOGLE_APPLICATION_CREDENTIALS')

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('proj_test_observe_data') \
    .set("spark.jars", jar_path) \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", creds_path)

### Only if an existing one already runs:
`sc.stop()`

In [8]:
sc = SparkContext(conf=conf)

22/10/25 15:16:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [9]:
hconf = sc._jsc.hadoopConfiguration()

hconf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hconf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hconf.set("fs.gs.auth.service.account.json.keyfile", creds_path)
hconf.set("fs.gs.auth.service.account.enable", "true")

In [10]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

***

### 1-time sample download for pandas reading to infer schema for everything else:
command:
`!wget https://data.cityofchicago.org/api/views/hx8q-mf9v/rows.csv?accessType=DOWNLOAD`

note: file is `Crimes_-_2012.csv`

output:
```
--2022-10-22 08:53:42--  https://data.cityofchicago.org/api/views/hx8q-mf9v/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.205, 52.206.68.26, 52.206.140.199
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [          <=>       ]  75.99M  2.54MB/s    in 28s     

2022-10-22 08:54:11 (2.68 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [79677853]
```

### check count here: raw csv file
command:
`!wc -l rows.csv?accessType=DOWNLOAD`

output:
`336247 rows.csv?accessType=DOWNLOAD`

### Because of `TypeError: Can not merge type (pandas string to spark double) for 'Location Description', 'location' fields`
Commands:
```
cols = ['Case Number', 'Date', 'Block', 'IUCR', 'Primary Type', 'Description', 'Arrest', 'Domestic', 'Beat', 'Ward', 'FBI Code', 'X Coordinate', 'Y Coordinate', 'Year', 'Latitude', 'Longitude']
df_pandas = pd.read_csv('rows.csv?accessType=DOWNLOAD', nrows=100, usecols=cols)
```

Command:
`spark.createDataFrame(df_pandas).schema`

Output:
```
StructType([StructField('Case Number', StringType(), True), StructField('Date', StringType(), True), StructField('Block', StringType(), True), StructField('IUCR', StringType(), True), StructField('Primary Type', StringType(), True), StructField('Description', StringType(), True), StructField('Arrest', BooleanType(), True), StructField('Domestic', BooleanType(), True), StructField('Beat', LongType(), True), StructField('Ward', LongType(), True), StructField('FBI Code', StringType(), True), StructField('X Coordinate', DoubleType(), True), StructField('Y Coordinate', DoubleType(), True), StructField('Year', LongType(), True), StructField('Latitude', DoubleType(), True), StructField('Longitude', DoubleType(), True)])
```

In [11]:
# modified pattern from pandas schema
schema_template = types.StructType([
    types.StructField('Case Number', types.StringType(), True),
    types.StructField('Date', types.StringType(), True),
    types.StructField('Block', types.StringType(), True),
    types.StructField('IUCR', types.StringType(), True),
    types.StructField('Primary Type', types.StringType(), True),
    types.StructField('Description', types.StringType(), True),
    types.StructField('Location Description', types.StringType(), True),
    types.StructField('Arrest', types.BooleanType(), True),
    types.StructField('Domestic', types.BooleanType(), True),
    types.StructField('Beat', types.StringType(), True),
    types.StructField('Ward', types.IntegerType(), True),
    types.StructField('FBI Code', types.StringType(), True),
    types.StructField('X Coordinate', types.FloatType(), True),
    types.StructField('Y Coordinate', types.FloatType(), True),
    types.StructField('Year', types.IntegerType(), True),
    types.StructField('Latitude', types.FloatType(), True),
    types.StructField('Longitude', types.FloatType(), True),
    types.StructField('Location', types.StringType(), True)
])

### Replace below with me:
```
df_csv = spark.read \
    .option("header", "true") \
    .schema(schema_template) \
    .csv(f'{gcs_bkt}/raw/{city}/')
```

In [12]:
df_csv = spark.read \
    .option("header", "true") \
    .schema(schema_template) \
    .csv(f'{gcs_bkt}/raw/{city}/' + 'Crimes_-_2001.csv')

### check count here: original df
Command:
`df_csv.count()`

Output:
`485853`

### check count Jan: csv df, strdate col
Command:
```
df_time \
    .filter(F.split('Date', ' ').getItem(0).startswith('01')) \
    .count()
```
Output:
`38114`

### check date Jan1: csv df, strdate col
Command:
```
df_time \
    .filter(F.split('Date', ' ').getItem(0) == '01/01/2001') \
    .count()
```
Output:
`1825`

In [55]:
def parse_dt(dt_str):
    """
    parse datetime object from given date string
    """
    cities = ['Chicago', 'San Francisco', 'Los Angeles', 'Austin']
    f_cities = [c.replace(' ', '_').lower() for c in cities]
    if city == f_cities[0]:
        return pdl.from_format(dt_str, 'MM/DD/YYYY HH:mm:ss A')

parse_dt_udf = F.udf(parse_dt, returnType=types.TimestampType())

In [56]:
# filter out rows with null values in important columns
df_time = df_csv \
    .filter(F.col('Date').isNotNull() & F.col('IUCR').isNotNull()) \
    .withColumn('Timestamp', parse_dt_udf(df_csv.Date))

In [57]:
years_rows = df_time.dropDuplicates(['Year']).select('Year').collect()
years = [row.Year for row in years_rows]

### check parsed dates

Command: `df_time.head(10)`

Output: must contain datetimes

### check count here: non-null df
Command:
`df_time.count()`

Output:
`485853`

### check parsed years
Command:
`print(years)`

Output:
`[2001]`

### check date here: csv df, datetime col
Command:
```
df_time \
    .filter(F.month('Timestamp') == 1) \
    .groupBy(F.dayofmonth('Timestamp')) \
    .count() \
    .orderBy('count', ascending=False).show()
```
Output:
```
+---------------------+-----+
|dayofmonth(Timestamp)|count|
+---------------------+-----+
|                    1| 1825|
|                   12| 1353|
|                   15| 1312|
|                   13| 1311|
|                   26| 1296|
|                    6| 1290|
|                   17| 1288|
|                   18| 1278|
|                    5| 1267|
|                   20| 1256|
|                   16| 1251|
|                   23| 1250|
|                   10| 1237|
|                   11| 1228|
|                   27| 1215|
|                   19| 1214|
|                   30| 1211|
|                   31| 1189|
|                    9| 1184|
|                   25| 1183|
+---------------------+-----+
only showing top 20 rows
```

In [92]:
df_time_part = df_time \
    .repartition(2)

In [93]:
df_time_part \
    .filter(F.month('Timestamp') == 1) \
    .groupBy(F.dayofmonth('Timestamp')) \
    .count() \
    .orderBy('count', ascending=False).show()

+---------------------+-----+
|dayofmonth(Timestamp)|count|
+---------------------+-----+
|                    1| 1825|
|                   12| 1353|
|                   15| 1312|
|                   13| 1311|
|                   26| 1296|
|                    6| 1290|
|                   17| 1288|
|                   18| 1278|
|                    5| 1267|
|                   20| 1256|
|                   16| 1251|
|                   23| 1250|
|                   10| 1237|
|                   11| 1228|
|                   27| 1215|
|                   19| 1214|
|                   30| 1211|
|                   31| 1189|
|                    9| 1184|
|                   25| 1183|
+---------------------+-----+
only showing top 20 rows



In [ ]:
# selected columns for analysis
df_time_cols = ['Case Number', 'Timestamp', 'Block', 'Primary Type', 'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat', 'Latitude', 'Longitude']

for year in years:
    df = df_time.filter(F.col('Year') == year)
    for month in range(1, 13):
        df_filtered = df.filter(F.month('Timestamp') == month) \
            .select(df_time_cols) \
            .repartition(2)
        for col in df_time_cols:
            df_filtered = df_filtered.withColumnRenamed(col, col.lower().replace(' ', '_'))
        df_filtered.write.parquet(f'{gcs_bkt}/pq/{city}/{year}/{month}', mode='overwrite')

In [ ]:
sc.stop()

### test pq accuracy
Command:
```
df_pq = spark.read \
    .option("header", "true") \
    .parquet(f'{gcs_bkt}/pq/{city}/2001/1')
```

### check count here: pq df
Command: `df_pq.count()`

Output: `38114`

### check date Jan: pq df
Command:
```
df_pq \
    .filter(F.month('Timestamp') == 1).count()
```
Output: `1825`

### check date Jan1: pq df
Command:
```
df_pq \
    .filter(F.dayofmonth('Timestamp') == 1).count()
```
Output: `1825`

### check date here: pq df
Command:
```
df_pq \
    .filter(F.month('Timestamp') == 1) \
    .groupBy(F.dayofmonth('Timestamp')) \
    .count() \
    .orderBy('count', ascending=False).show()
```